In [49]:
from transformers import DistilBertModel,DistilBertTokenizer,BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re
from transformers import DistilBertModel, DistilBertTokenizer
from elasticsearch import Elasticsearch
import time
from elasticsearch.helpers import bulk
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from elasticsearch.helpers import bulk, BulkIndexError
import joblib

In [17]:
Elasticsearch.requestTimeout: 90000
max_length = 40 # bert 句子向量的最大值
input_length = 23 # torch.Size([44320, 23, 768]) 这个要跟中间的23长度对应（第四个代码框），因为description里面长度影响结果
n_components = 900 #PCA降维后的维度
movie_num = 44320 # movie number
# input_sentence means the description user input
# host is:'http://localhost:9200'

# 需要保存三个文件 第一个是pca矩阵 第二个是normalized的最大值最小值数组

In [3]:
# tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
# model = BertModel.from_pretrained('distilbert-base-uncased')#bert-base-uncased
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


movie_data = pd.read_csv('C:\\Users\\16500\\Desktop\\COMP7705 Project\\Dataset\\m.csv')

# A function to remove punctuation marks
def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)  # Remove Chinese garbled characters
    return cleaned_text

# Remove punctuation from the description
text = movie_data['overview'].astype(str).apply(remove_punctuation)
title = movie_data['title'].astype(str)

# Define the text to be encoded
input_text = text.copy()

# If the text length is less than the maximum length, it is filled
input_text = input_text.apply(lambda x: x[:max_length-2])  # Consider adding the length of the start and end tags
input_text = input_text.apply(lambda x: '[CLS]' + x + '[SEP]')

# Word segmentation and encoding of text using a tokenizer
tokens = tokenizer.batch_encode_plus(
    input_text.tolist(),
    padding='longest',
    truncation=True,
    max_length=max_length,
    return_tensors='pt'
)

# Input the encoded text into the BERT model to get the output
with torch.no_grad():
    outputs = model(**tokens)

# Gets the output of the last hidden layer as an embedded representation of the word
embeddings = outputs.last_hidden_state

# Print the embedded representation of the first word
print(embeddings[0])



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\16500\AppData\Local\Temp\ipykernel_1880\3202862208.py:5: DtypeWarning: Columns (0,1,3,6,38,41,42,43,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv('C:\\Users\\16500\\Desktop\\COMP7705 Projec

tensor([[-0.1813,  0.0574,  0.1665,  ..., -0.0052,  0.1270,  0.1243],
        [ 0.0735, -0.0664,  0.2780,  ...,  0.0308,  0.1524,  0.0677],
        [-0.0502,  0.1476,  0.0210,  ...,  0.2464,  0.0771, -0.2482],
        ...,
        [ 0.1926, -0.1003,  0.0791,  ...,  0.3455,  0.1029, -0.1931],
        [ 0.1716, -0.1461,  0.1154,  ...,  0.3566,  0.1148, -0.2806],
        [ 0.1566, -0.1967,  0.1446,  ...,  0.3273,  0.1331, -0.3121]])


In [6]:
embeddings.shape

torch.Size([44320, 23, 768])

In [9]:
# e = embeddings.reshape(998, -1) # transform to 1*14592 
e = embeddings.reshape(movie_num, -1) # transform to 1*14592 


In [10]:
# Convert a floating point tensor to a numpy array
A = e.numpy()
print(A.shape)

# Create PCA models and fit the data

pca = PCA(n_components=n_components)
pca.fit(A)

# Performing PCA dimension reduction
embedding_pca = pca.transform(A)

# The result of dimensionality reduction was normalized
scaler1 = MinMaxScaler(feature_range=(-1, 1))
embedding_pca_scaled = scaler1.fit_transform(embedding_pca)
min_val = scaler1.data_min_
max_val = scaler1.data_max_
# print(min_val,max_val)
# print(embedding_pca_scaled.shape)

# Save the minimum and maximum values as parameters
params = {'min_val': min_val, 'max_val': max_val}

(44320, 17664)


In [51]:
joblib.dump(pca, 'pca_model.pkl')
print("PCA save to pca_model.pkl")
pca = joblib.load('pca_model.pkl')
pca

PCA save to pca_model.pkl


PCA(n_components=900)

In [60]:
# print((min_val.shape))
# print(min_val,max_val)
min_val = scaler1.data_min_
max_val = scaler1.data_max_
np.save('min_val.npy', min_val)
np.save('max_val.npy', max_val)
min_val_load = np.load('min_val.npy')
max_val_load = np.load('max_val.npy')


In [11]:
title = title.tolist()
description = text.tolist()
embedding_pca_scaled_list = embedding_pca_scaled.tolist()
embedding_pca_scaled[0]

array([-8.82123590e-01, -2.64966071e-01, -5.98165214e-01, -2.34743312e-01,
        3.04510385e-01, -6.67465031e-01, -8.58974084e-02, -1.06428728e-01,
       -2.23919094e-01, -3.40628475e-01, -2.94826090e-01, -1.89420953e-01,
       -3.94587636e-01, -4.06518430e-01,  4.79652673e-01, -1.13538772e-01,
       -2.14205518e-01, -4.20148037e-02, -4.19248164e-01, -8.45535845e-03,
       -2.46765122e-01, -1.16412655e-01, -1.41649038e-01, -3.68033238e-02,
       -2.44288817e-02, -1.94472224e-01, -2.12964863e-01, -4.94941361e-02,
       -3.42215061e-01, -2.07310189e-02,  1.48657516e-01, -1.93170190e-01,
        2.16150954e-02, -1.57360151e-01, -9.07145813e-02, -1.22617334e-01,
        1.05403319e-01, -2.38230050e-01,  1.18965298e-01, -2.32061028e-01,
       -2.28489891e-01, -9.23217535e-02, -1.54602125e-01, -1.89369842e-01,
        9.39393640e-02, -5.25224328e-01,  1.76450573e-02, -8.46208185e-02,
        3.70062441e-02, -1.04545891e-01, -4.80969846e-01,  3.87251377e-02,
       -9.39062461e-02, -

In [12]:
# Create Elasticsearch client
es = Elasticsearch(hosts='http://localhost:9200')

# Create an index map
index_mapping = {
    "settings": {
        "index": {
            "number_of_shards": "1",
            "number_of_replicas": "0"
        }
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "overview":{"type": "text"},
            "vector": {"type": "dense_vector", "dims": 900}
        }
    }
}

#  CREATE INDEX 
es.indices.create(index="movies", body=index_mapping, ignore=[400, 404])

# Create the document data to index
docs = []
for idx, movie_title in enumerate(title):
    embedding = embedding_pca_scaled[idx].tolist()
    overview = description[idx]
    d = {
        "_index": "movies",  # Assign index name
        "title": movie_title,
        "overview":overview,
        "vector": embedding
    }
    docs.append(d)
    
# Batch index document data to Elasticsearch
success, _ = bulk(es, docs, index="movies", chunk_size=998, request_timeout=120)
print(f"Successfully indexed {success} movies.")

C:\Users\16500\AppData\Local\Temp\ipykernel_1880\2389479420.py:22: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="movies", body=index_mapping, ignore=[400, 404])
C:\Users\16500\AppData\Local\Temp\ipykernel_1880\2389479420.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="movies", body=index_mapping, ignore=[400, 404])
C:\Users\16500\AppData\Local\Temp\ipykernel_1880\2389479420.py:38: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, _ = bulk(es, docs, index="movies", chunk_size=998, request_timeout=120)


Successfully indexed 44320 movies.


In [72]:
start_time = time.time()

In [73]:
input_sentence = "The man had hidden an important microfilm in his passport which the thieves pass onto a forger friend Panadol Tsui Hark Inspector Ng Michelle Yeoh is assigned to the case along with Inspector Morris Cynthia Rothrock from Scotland Yard"

input_sentence = remove_punctuation(input_sentence)

input_tokens = tokenizer.encode_plus(
    input_sentence,
    truncation=True,
    max_length=input_length - 2,  # Consider adding the length of the start and end tags
    padding='max_length',
    return_tensors='pt'
)

input_ids = input_tokens['input_ids']  # Enter the encoding of the sentence

# Add start and end tags to the beginning and end of the input sentence
if len(input_ids) < input_length:
    input_ids = torch.cat([torch.tensor([tokenizer.cls_token_id]), input_ids.squeeze(), torch.tensor([tokenizer.sep_token_id])])
#     attention_mask = torch.cat([torch.tensor([1]), attention_mask.squeeze(), torch.tensor([1])])

# If the length of the input sentence is less than input length, fill it in
if len(input_ids) < input_length:
    num_pad_tokens = input_length - len(input_ids)
    input_ids = F.pad(input_ids, (0, num_pad_tokens), value=tokenizer.pad_token_id)
#     attention_mask = F.pad(attention_mask, (0, num_pad_tokens), value=0)

# Converts the input to a tensor and gets an embedded representation
input_ids = input_ids.unsqueeze(0)
# attention_mask = attention_mask.unsqueeze(0)

with torch.no_grad():
    input_outputs = model(input_ids)
    input_embedding = input_outputs.last_hidden_state.squeeze(0)

In [74]:
embedding_np = input_embedding.detach().numpy().reshape(-1).tolist()
len(embedding_np)

17664

In [75]:
# Convert embedding_np to a NumPy array
embedding_np_array = np.array(embedding_np)

# The transformation was performed using the previously trained PCA model
input_pca = pca.transform(embedding_np_array.reshape(1, -1))

input_pca.shape
# Use MinMaxScaler for standardization
scaler2 = MinMaxScaler(feature_range=(-1, 1))
scaler2.data_min_ = min_val
scaler2.data_max_ = max_val

normalized_data = (input_pca - min_val) / (max_val - min_val)

# Convert the result to a 1x900 list
normalized_data_list = np.squeeze(normalized_data).tolist()

print(normalized_data_list)

# scaler1

[0.438738944442068, 0.6740272632354265, 0.5261202324720423, 0.4750907450772413, 0.31878604143621364, 0.521155912720735, 0.372691333545024, 0.43191353376260094, 0.5838327089555898, 0.4904514895219783, 0.6356202302069138, 0.4532993722797324, 0.40384279080232943, 0.42376847023218434, 0.23154329160950718, 0.5065956953957103, 0.48416330332713503, 0.5298578301281954, 0.47777267618702035, 0.4892678911196422, 0.4518728274148447, 0.45660248433078515, 0.5296893619054487, 0.5696997908111625, 0.5573027530304772, 0.28702913052731394, 0.5983358914622977, 0.5138929833365283, 0.3622364307334524, 0.29815034728108397, 0.41476806193055205, 0.6443493376633881, 0.38544450293396165, 0.48465190368237043, 0.44228621471718793, 0.5888205559446833, 0.4712533969888025, 0.2803308101684022, 0.11740428342330138, 0.39591389023319357, 0.3778650529027232, 0.3868670645672167, 0.25830567718453584, 0.3804373779269912, 0.5187151893830273, 0.36955072184854665, 0.7597245849701436, 0.6064798553199853, 0.4229367598448739, 0.48

In [76]:
def search_similar_movies(query_vector, top_k):
    body = {
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        },
        "_source": ["title","overview"],
        "size": top_k
    }

    response = es.search(index='movies', body=body)
    similar_movies = [hit['_source']['title'] for hit in response['hits']['hits']]
    description = [hit['_source']['overview'] for hit in response['hits']['hits']]
    return similar_movies,description

# Assuming you already have query_vec tor, you can use the following code to invoke the function that searches for similar movies
similar_movies,description = search_similar_movies(normalized_data_list, top_k=20)
for i, j in zip(similar_movies,description):
    print("Title: "+ i + " Description: "+j)
    print("")

# print(description)


Title: Symmetry Description: Dwudziestoszeioletni kasz niessznie poszony o napad na starszkobiet trafia do aresztu edczego Adwokat wynajy przez matkbezskutecznie pruje go stamt wydosta Pod wpwem dramatycznych wydarzew celi przejiowej kasz za wszelkcenchce znale siw grypsujej czci wini iat git wymaga jednak przestrzegania panujych tu twardych zasad Reser uzyskazgodMinistra Spraw Wewnrznych na realizacjzdj w wiieniu na Biace

Title: Komedia malzenska Description: Maetwo Marii i Wiktora przechodzi kryzys Kobieta postanawia pokazamowi jak trudna jest praca w domu dlatego wyje do innego miasta Jak poradzi sobie Wiktor z ziei i wszystkimi obowikami Jak Maria da sobie radw wielkim mieie 

Title: Jasminum Description: Dzieje siwspczeie Jasminum cika nazwa jainu opowiada historitajemniczi magiczn pezmyswych zapach wypeionszczegnym iatm jakie roztacza wok siebie mi Klasztor w ktym toczy siakcja filmu nie rniy siod innych gdyby nie fakt  trzech mieszkajych tam zakonnik posiada pewnniezwyk cech Ka

C:\Users\16500\AppData\Local\Temp\ipykernel_1880\3624737538.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index='movies', body=body)


In [77]:
end_time = time.time()
print("Time elapsed:", end_time - start_time, "seconds")

Time elapsed: 2.1225385665893555 seconds
